# **BERTopic on German & French Amazon Reviews**

In our previous experiment, we proved that **KeyBERT-Inspired representations** significantly outperform standard frequency-based labels (c-TF-IDF) for English news. They produce cleaner, more semantic topic descriptions.

In this notebook, we test if that same strategy holds up in a multilingual environment using **Amazon Reviews** in **German** and **French**. Reviews are \"messy\" data filled with slang, typos, and strong sentiment. We will train separate BERTopic models for each language and apply the KeyBERT-inspired representation to see if we can extract clear, interpretable product categories from raw customer feedback.


In [38]:
!pip install --upgrade pip
!pip install -r requirements.txt
!python -m spacy download de_core_news_lg
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 9.1 MB/s  0:00:580:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 8.8 MB/s  0:01:020:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [86]:
from datasets import load_dataset

ds_german = load_dataset("SetFit/amazon_reviews_multi_de")
ds_french = load_dataset("SetFit/amazon_reviews_multi_fr")

docs_train_german = ds_german["train"]["text"]
docs_train_french = ds_french["train"]["text"]

Repo card metadata block was not found. Setting CardData to empty.


### Data Preparation
Based on the results of the previous experiment, we decided to use **lemmatization** this time to improve BERTopic. We did so because of the following reasons:
- we can remove the noise (words like Produkte, Produkten, Produktes will be merged into one -> Produkt)
- topics should be more semantically consistent (because of fewer repeated word variants)
- smaller vocabulary means more stable clustering

In [87]:
import spacy
import random

nlp_de = spacy.load("de_core_news_lg")
nlp_fr = spacy.load("fr_core_news_lg")

random.seed(42)
docs_sample_german = random.sample(docs_train_german, 10000)
docs_sample_french = random.sample(docs_train_french, 10000)

def lemmatize(text: str, lang: str):
    if lang == "de":
        doc = nlp_de(text)
    else:
        doc = nlp_fr(text)
    return [tok.lemma_ for tok in doc if tok.is_alpha]

lemmatized_german = [lemmatize(text, "de") for text in docs_sample_german]
lemmatized_french = [lemmatize(text, "fr") for text in docs_sample_french]

Original German: Armband ist leider nach 1 Jahr kaputt gegangen
Lemmatized German: ['der', 'Glitzer', 'kommen', 'doppelt', 'verpacken', 'wer', 'sehr', 'ordentlich', 'sein', 'damit', 'man', 'kein', 'böes', 'Überraschung', 'bei', 'öffnen', 'erleben', 'ich', 'haben', 'nachdem', 'ich', 'diverser', 'Rezension', 'lesen', 'haben', 'der', 'Packung', 'in', 'Liter', 'Farbe', 'mischen', 'und', 'DAS', 'REICHT', 'es', 'glitzern', 'angenehm', 'und', 'man', 'werden', 'nicht', 'erschlagen', 'der', 'Farbe', 'lässt', 'sich', 'einwandfrei', 'streichen', 'man', 'sollen', 'lediglich', 'mehrfach', 'mit', 'der', 'Rolle', 'über', 'ein', 'Stelle', 'gehen', 'um', 'der', 'Glitzer', 'gleichmäßig', 'zu', 'verteilen', 'ein', 'klein', 'Manko', 'geben', 'es', 'dafür', 'sein', 'allerdings', 'kein', 'Abzug', 'an', 'Stern', 'notwendig', 'der', 'ganz', 'Wohnung', 'glitzern', 'ein', 'der', 'erster', 'Zeit', 'da', 'sich', 'der', 'Partikel', 'leicht', 'verteilen']
Original French: A déconseiller - Article n'a fonctionné qu'

In [88]:


def remove_stopwords(lemmas: list[str], lang: str) -> str:
    if lang == "de":
        stopwords = nlp_de.Defaults.stop_words
    elif lang == "fr":
        stopwords = nlp_fr.Defaults.stop_words
    else:
        raise ValueError("lang must be 'de' or 'fr'")

    cleaned = [lemma for lemma in lemmas if lemma.lower() not in stopwords]
    return " ".join(cleaned)

docs_cleaned_german = [remove_stopwords(lemmas, "de") for lemmas in lemmatized_german]
print("Cleaned German:", docs_cleaned_german[0])

docs_cleaned_french = [remove_stopwords(lemmas, "fr") for lemmas in lemmatized_french]
print("Cleaned French:", docs_cleaned_french[0])

Cleaned German: Glitzer doppelt verpacken ordentlich böes Überraschung öffnen erleben diverser Rezension lesen Packung Liter Farbe mischen REICHT glitzern angenehm erschlagen Farbe lässt einwandfrei streichen lediglich mehrfach Rolle Stelle Glitzer gleichmäßig verteilen klein Manko geben Abzug Stern notwendig Wohnung glitzern Partikel verteilen
Cleaned French: ressembler image afficher ici boîte outr trouver taille réel clairement indiquer


### **Set up**
##### **Baseline BERTopic model for German reviews**

Here, we train a BERTopic model on German Amazon reviews. We use the multilingual embedding setting and restrict the topic categorization to 10 topics with a minimum topic size of 20 documents.

In [98]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-mpnet-base-v2")
vectorizer_model = CountVectorizer(stop_words="english")
representation_model_chain = [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.4)]

In [99]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

topic_model_german = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,
    nr_topics=10,
    representation_model=representation_model_chain
)

topics_base_german, probs_base_german = topic_model_german.fit_transform(docs_cleaned_german)

2025-12-07 13:28:13,792 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 313/313 [05:28<00:00,  1.05s/it]
2025-12-07 13:33:43,770 - BERTopic - Embedding - Completed ✓
2025-12-07 13:33:43,807 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 13:33:49,592 - BERTopic - Dimensionality - Completed ✓
2025-12-07 13:33:49,609 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 13:33:53,021 - BERTopic - Cluster - Completed ✓
2025-12-07 13:33:53,022 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-07 13:33:53,748 - BERTopic - Representation - Completed ✓
2025-12-07 13:33:53,749 - BERTopic - Topic reduction - Reducing number of topics
2025-12-07 13:33:53,827 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 13:34:24,997 - BERTopic - Representation - Completed ✓
2025-12-07 13:34:25,209 - BERTopic - Topic reduction - Re

##### **Baseline BERTopic model for French reviews**

We repeat the same setup for the French Amazon reviews, again using a multilingual model and the KeyBERTInspired representation.

In [103]:
representation_model_french = KeyBERTInspired()

topic_model_french = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,
    nr_topics=10,
    representation_model=representation_model_chain
)

topics_base_french, probs_base_french = topic_model_french.fit_transform(docs_cleaned_french)

2025-12-07 13:35:35,940 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 313/313 [03:02<00:00,  1.72it/s]
2025-12-07 13:38:38,830 - BERTopic - Embedding - Completed ✓
2025-12-07 13:38:38,835 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 13:38:44,338 - BERTopic - Dimensionality - Completed ✓
2025-12-07 13:38:44,383 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 13:38:47,082 - BERTopic - Cluster - Completed ✓
2025-12-07 13:38:47,083 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-07 13:38:47,854 - BERTopic - Representation - Completed ✓
2025-12-07 13:38:47,855 - BERTopic - Topic reduction - Reducing number of topics
2025-12-07 13:38:47,885 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 13:42:30,110 - BERTopic - Representation - Completed ✓
2025-12-07 13:42:30,231 - BERTopic - Topic reduction - Re

### **Inspecting computed topics for German**

In [100]:
info_df = topic_model_german.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(6))

,Topic,Count,Top Keywords
1,0,5684,"lieferung, beschreiben, schnell, bild, größe, produkt"
2,1,560,"adapter, bluetooth, funktionieren, usb, headset, smartphone"
3,2,199,"film, montageanleitung, dvd, leistung, langweilig, lieferung"
4,3,147,"geschichte, lieferung, langweilig, kinderleicht, meinung, kinderspielzeug"
5,4,90,"zufrieden, zeitlich, zubehör, zeitaufwendig, vorzelt, frechheit"
6,5,82,"armband, armbänd, armbanduhr, fitnessband, arm, leistung"


The model successfully identifies specific product categories despite the noisy nature of review data. We see clear clusters for bluetooth and adapter functionality (`bluetooth`, `usb`), or movies (`film`, `dvd`).

However, we see the limitations of working with raw review data. Unlike the clean news dataset, these topics are polluted by **sentiment words** (`langweilig`) . KeyBERT captures the semantic core of the topic and produces domain-specific topics. 


#### **Visualization of German topic space**

We visualize the German topics in a 2D projection to see how well-separated they are. In the bar chart, we can inspect particular words in those clusters along with their significance.

In [101]:
fig_map = topic_model_german.visualize_topics()
fig_map.show()

fig_bar = topic_model_german.visualize_barchart(top_n_topics=10)
fig_bar.show()

#### **Inspecting computed French topics**

We repeat the same steps for the inspection of the topics generated by the French model by building a clean topic overview table with interpretable keyword representations.

In [104]:
info_df = topic_model_french.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(6))

,Topic,Count,Top Keywords
1,0,4516,"vraiment, qualité, pratique, contre, pouvoir, bon"
2,1,985,"installer, correctement, chargeur, installation, écouteur, recevoir"
3,2,55,"bracelet, anniversaire, poignet, recevoir, parfaitement, vraiment"
4,3,51,"nickel, coincer, coin, cheminée, baterie, rien"
5,4,50,"sympa, estompe, vraiment, histoire, décevoir, voir"
6,5,48,"led, lampe, circuit, phosphorescence, halogène, changement"


The French model demonstrates a strong ability to identify specific hardware niches, even within a noisy dataset. We see distinct, high-quality clusters for bracelets and wrist accessories (as gifts) (Topic 2: `bracelet`, `poignet`) and lighting (Topic 4: `lampe`, `halogène`).

However, the noise in the dataset is maybe even more aggressive here than in the German dataset. Topic 0 and Topic 1 are very general clusters dominated by overlapping not very domain-specific words like `vraiment` (really), `pouvoir` (can), and `recevoir` (receive). Unlike standard nouns, these words don't describe a product; they describe the act of reviewing.

#### **Visualization of the French topic space**

Here, we visualize the French topic landscape using the same projections and bar chart as for the German topics. This allows us to compare it qualitatively to the German topics.

In [105]:
fig_map = topic_model_french.visualize_topics()
fig_map.show()

fig_bar = topic_model_french.visualize_barchart(top_n_topics=10)
fig_bar.show()

In [106]:
from calculate_t_coherence_and_diversity import evaluate_bertopic_pmi

_, npm_german, div_german = evaluate_bertopic_pmi(
    topic_model=topic_model_german,
    docs=docs_cleaned_german,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="German"
)

_, npm_french, div_french = evaluate_bertopic_pmi(
    topic_model=topic_model_french,
    docs=docs_cleaned_french,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="French"
)

results_df = pd.DataFrame({
    "Metric": ["Coherence (NPMI)", "Diversity"],
    "German": [npm_german, div_german],
    "French": [npm_french, div_french],
})
display(results_df)

German Model - NPMI: 0.2670
German Model - Diversity: 0.3511
French Model - NPMI: 0.1612
French Model - Diversity: 0.3156


,Metric,German,French
0,Coherence (NPMI),0.266967,0.161227
1,Diversity,0.351111,0.315556


# Evaluation

This project aimed to replicate the success of our English News experiment on a more challenging, multilingual dataset of Amazon Reviews. While we successfully extracted interpretable product categories for both German and French, the quantitative metrics reveal that it was far less successful. The coherence scores for both languages are between **0.26 and 0.16**, which is significantly lower than the **~0.40** we achieved with the BBC News dataset.

These lower scores do not necessarily mean the model failed, they rather highlight the nature of the review data. Reviews are short, informal, and heavily saturated with sentiment (e.g., "bad", "perfect") which makes them harder to work with in topics extraction. While the KeyBERT-Inspired representation successfully identified specific items like "LEDs" it could not fully overcome the noise of raw text.

To bridge the gap between the score of **0.26** and our previous benchmark of **0.40**, future iterations with customized lemmatization and stop word removal preprocessing steps could potentially improve the results.